In [1]:
import re
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import r2_score
from joblib import Parallel, delayed
from itertools import combinations_with_replacement
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, DotProduct, ExpSineSquared, Matern, RBF, RationalQuadratic, WhiteKernel

### Dataset

In [2]:
data = f"dataset/dataset-essential/constant_series.csv"
df = pd.read_csv(data)
X = df['value'].to_numpy()

### Functions

In [3]:
def substitute_constant_kernel(kernels_list):
    for idx, kernel in enumerate(kernels_list):
        kernel_str = str(kernel)
        kernel_str = re.sub(r'1\*\*2', 'ConstantKernel()', kernel_str)
        kernels_list[idx] = kernel_str
    return kernels_list

In [4]:
def is_pos_def(x):
  return np.all(np.linalg.eigvals(x) > 0) and np.allclose(x, x.T)

In [5]:
def process_kernel_combination(kernel_str, X):
  try:
    from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, WhiteKernel
    kernel_combinado = eval(kernel_str)
    matriz_cov_combinacao_kernels = kernel_combinado(X)

    if is_pos_def(matriz_cov_combinacao_kernels):
      return kernel_combinado

  except TypeError:
    pass

  return None

### Combinations

In [6]:
kernels_k = []
kernels_y = []
kernels_total = []

In [7]:
kernels = ['ConstantKernel()', 'DotProduct()', 'ExpSineSquared()', 'Matern()', 'RBF()', 'RationalQuadratic()', 'WhiteKernel()']
operations = ['*', '+']

##### 1. Combinations k1

In [8]:
for i in kernels:
  kernels_k.append(eval(i))
  kernels_y.append(eval(i))

##### 2. Combinations k1_k2

In [9]:
n_k = 2
n_op = 1

two_elements = list(combinations_with_replacement(kernels, n_k))
combinations_two = [f"{elem1} {op} {elem2}" for elem1, elem2 in two_elements for op in operations]

2.1 Validate kernel k1_k2

In [10]:
results_two = Parallel(n_jobs=-1)(delayed(process_kernel_combination)(element, X) for element in tqdm(combinations_two, desc='Progress', unit='kernels', position=0, leave=True))

Progress: 100%|██████████| 56/56 [00:01<00:00, 34.61kernels/s]


In [11]:
kernels_k.extend([result for result in results_two if result is not None])

##### 3. Combinations k1_k2_k3

In [12]:
n_k = 3
n_op = 2

three_elements = list(combinations_with_replacement(kernels, n_k))
two_operations = list(combinations_with_replacement(operations, n_op))
combinations_three = [f"{elem1} {op1} {elem2} {op2} {elem3}" for elem1, elem2, elem3 in three_elements for op1, op2 in two_operations]

3.1 Validate Kernel k1_k2_k3

In [13]:
results_three = Parallel(n_jobs=-1)(delayed(process_kernel_combination)(element, X) for element in tqdm(combinations_three, desc='Progress', unit='kernels', position=0, leave=True))

Progress: 100%|██████████| 252/252 [00:00<00:00, 4618.95kernels/s]


In [14]:
kernels_k.extend([result for result in results_three if result is not None])

##### 4. Combinations k1_k2_k3_k4

In [15]:
n_k = 4
n_op = 3

four_elements = list(combinations_with_replacement(kernels, n_k))
three_operations = list(combinations_with_replacement(operations, n_op))
combinations_four = [f"{elem1} {op1} {elem2} {op2} {elem3} {op3} {elem4}" for elem1, elem2, elem3, elem4 in four_elements for op1, op2, op3 in three_operations]

4.1 Validate kernel k1_k2_k3_k4

In [16]:
results_four = Parallel(n_jobs=-1)(delayed(process_kernel_combination)(element, X) for element in tqdm(combinations_four, desc='Progress', unit='kernels', position=0, leave=True))

Progress: 100%|██████████| 840/840 [00:00<00:00, 8156.29kernels/s]


In [17]:
kernels_k.extend([result for result in results_four if result is not None])

##### 5. Combinations k1_k2_k3_k4_k5

In [18]:
n_k = 5
n_op = 4

five_elements = list(combinations_with_replacement(kernels, n_k))
four_operations = list(combinations_with_replacement(operations, n_op))
combinations_five = [f"{elem1} {op1} {elem2} {op2} {elem3} {op3} {elem4} {op4} {elem5}" for elem1, elem2, elem3, elem4, elem5 in five_elements for op1, op2, op3, op4 in four_operations]

5.1 Validate kernel k1_k2_k3_k4_k5

In [19]:
results_five = Parallel(n_jobs=-1)(delayed(process_kernel_combination)(element, X) for element in tqdm(combinations_five, desc='Progress', unit='kernels', position=0, leave=True))

Progress: 100%|██████████| 2310/2310 [00:00<00:00, 11190.84kernels/s]


In [20]:
kernels_k.extend([result for result in results_five if result is not None])

##### 6. Combinations k1_k2_k3_k4_k5_k6

In [21]:
n_k = 6
n_op = 5

six_elements = list(combinations_with_replacement(kernels, n_k))
five_operations = list(combinations_with_replacement(operations, n_op))
combinations_six = [f"{elem1} {op1} {elem2} {op2} {elem3} {op3} {elem4} {op4} {elem5} {op5} {elem6}" for elem1, elem2, elem3, elem4, elem5, elem6 in six_elements for op1, op2, op3, op4, op5 in five_operations]

6.1 Validate kernel k1_k2_k3_k4_k5_k6

In [22]:
results_six = Parallel(n_jobs=-1)(delayed(process_kernel_combination)(element, X) for element in tqdm(combinations_six, desc='Progress', unit='kernels', position=0, leave=True))

Progress: 100%|██████████| 5544/5544 [00:00<00:00, 13449.94kernels/s]


In [23]:
kernels_k.extend([result for result in results_six if result is not None])

##### 7. Combinations k1_k2_k3_k4_k5_k6_k7

In [24]:
n_k = 7
n_op = 6

seven_elements = list(combinations_with_replacement(kernels, n_k))
six_operations = list(combinations_with_replacement(operations, n_op))
combinations_seven = [f"{elem1} {op1} {elem2} {op2} {elem3} {op3} {elem4} {op4} {elem5} {op5} {elem6} {op6} {elem7}" for elem1, elem2, elem3, elem4, elem5, elem6, elem7 in seven_elements for op1, op2, op3, op4, op5, op6 in six_operations]

7.1 Validate kernel k1_k2_k3_k4_k5_k6_k7

In [25]:
results_seven = Parallel(n_jobs=-1)(delayed(process_kernel_combination)(element, X) for element in tqdm(combinations_seven, desc='Progress', unit='kernels', position=0, leave=True))

Progress: 100%|██████████| 12012/12012 [00:00<00:00, 13825.91kernels/s]


In [26]:
kernels_k.extend([result for result in results_seven if result is not None])

### Generate File with all kernel combinations

In [27]:
substituted_kernels_k = substitute_constant_kernel(kernels_k)

In [28]:
file_name = f"dataset/dataset_possibilities/constant_series_kernel_possibilities.csv"

In [29]:
with open(file_name, 'w', newline='') as file_csv:
    writer = csv.writer(file_csv)
    writer.writerow(['kernel_str'])
    for kernel in kernels_k:
        writer.writerow([kernel])

print(f"Data saved in {file_csv}")

Data saved in <_io.TextIOWrapper name='dataset/dataset_possibilities/constant_series_kernel_possibilities.csv' mode='w' encoding='cp1252'>
